In [17]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix

In [18]:
img_shape = 128
input_shape = (img_shape, img_shape, 3)
EPOCHS = 10
BATCH_SIZE = 32

In [19]:
def process_path(file_path):
    label = tf.strings.split(file_path, os.path.sep)[-2]
    label_arr = None
    if label == 'fake':
        label_arr = 0
    else:
        label_arr = 1
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [img_shape, img_shape])
    img = tf.image.convert_image_dtype(img, tf.float32)
    return (img, label_arr)

def scale(image, label):
    return (image / 255, label)

train_ds = tf.data.Dataset.list_files('1frame/train/*/*')
test_ds = tf.data.Dataset.list_files('1frame/test/*/*')
val_ds = tf.data.Dataset.list_files('1frame/valid/*/*')

train_size = len(train_ds)
val_size = len(val_ds)

train_ds = train_ds.shuffle(train_size).map(process_path, num_parallel_calls = tf.data.AUTOTUNE).map(scale, num_parallel_calls = tf.data.AUTOTUNE).batch(BATCH_SIZE).repeat(EPOCHS).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(val_size).map(process_path, num_parallel_calls = tf.data.AUTOTUNE).map(scale, num_parallel_calls = tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.shuffle(val_size).map(process_path, num_parallel_calls = tf.data.AUTOTUNE).map(scale, num_parallel_calls = tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [20]:
from tensorflow.keras.applications import InceptionResNetV2, EfficientNetB0, ResNet50
from tensorflow.keras.layers import Conv2D, BatchNormalization, Input, LeakyReLU, Concatenate
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, TimeDistributed, LSTM
from tensorflow.keras.layers import InputLayer, Rescaling
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [21]:
resnet = ResNet50(include_top = False, weights = 'imagenet', input_shape = input_shape)
resnet.trainable = True
model = Sequential()
model.add(resnet)
model.add(Dense(units = 16, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 8, activation = 'relu'))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(units = 8, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))

In [22]:
model.compile(loss = 'binary_crossentropy', optimizer = optimizers.Adam(learning_rate = 1e-3), metrics = ['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 4, 4, 2048)        23587712  
_________________________________________________________________
dense_16 (Dense)             (None, 4, 4, 16)          32784     
_________________________________________________________________
dropout_8 (Dropout)          (None, 4, 4, 16)          0         
_________________________________________________________________
dense_17 (Dense)             (None, 4, 4, 8)           136       
_________________________________________________________________
time_distributed_4 (TimeDist (None, 4, 32)             0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)              

In [ ]:
checkpoint_callback = ModelCheckpoint(
    filepath = 'checkpoints/best_1fdfd_resnet_lstm_dfdc.hdf5',
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True,
    mode = 'min')

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', 
                              factor = 0.2,
                              patience = 3, 
                              min_lr = 0.0001)

STEPS = train_size // BATCH_SIZE
callbacks = [checkpoint_callback, reduce_lr]
history = model.fit(train_ds, batch_size = BATCH_SIZE, steps_per_epoch = STEPS, epochs = EPOCHS, callbacks = callbacks, validation_data = val_ds, verbose = 1)

Epoch 1/10
4894/4894 [==============================] - 574s 116ms/step - loss: 0.6187 - accuracy: 0.6562 - val_loss: 0.6365 - val_accuracy: 0.6543

Epoch 00001: val_loss improved from inf to 0.63647, saving model to checkpoints\best_1fdfd_resnet_lstm_dfdc.hdf5
Epoch 2/10
4894/4894 [==============================] - 604s 123ms/step - loss: 0.5163 - accuracy: 0.7477 - val_loss: 0.6275 - val_accuracy: 0.6712

Epoch 00002: val_loss improved from 0.63647 to 0.62750, saving model to checkpoints\best_1fdfd_resnet_lstm_dfdc.hdf5
Epoch 3/10
4894/4894 [==============================] - 602s 123ms/step - loss: 0.3856 - accuracy: 0.8311 - val_loss: 0.9119 - val_accuracy: 0.5652

Epoch 00003: val_loss did not improve from 0.62750
Epoch 4/10
4894/4894 [==============================] - 544s 111ms/step - loss: 0.2872 - accuracy: 0.8886 - val_loss: 1.0227 - val_accuracy: 0.5543

Epoch 00004: val_loss did not improve from 0.62750
Epoch 5/10
4894/4894 [==============================] - 533s 109ms/step 

In [ ]:
model.save('models/resnet_lstm_dfd_model_1fdfd')

f, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 4))
t = f.suptitle('EfficientNetB0 with LSTM on Kaggle Dataset', fontsize = 12)
f.subplots_adjust(top = 0.85, wspace = 0.3)

epoch_list = list(range(1, EPOCHS + 1))
ax1.plot(epoch_list, history.history['accuracy'], label = 'Train Accuracy')
ax1.plot(epoch_list, history.history['val_accuracy'], label = 'Validation Accuracy')
ax1.set_xticks(np.arange(0, EPOCHS + 1, 1))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch #')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label = 'Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label = 'Validation Loss')
ax2.set_xticks(np.arange(0, EPOCHS + 1, 1))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch #')
ax2.set_title('Loss')
l2 = ax2.legend(loc = "best")

In [ ]:
results = model.evaluate(test_ds)
print(model.metrics_names)
print(results)

In [ ]:
#Output confusion matrix
def print_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    print('True positive = ', cm[0][0])
    print('False positive = ', cm[0][1])
    print('False negative = ', cm[1][0])
    print('True negative = ', cm[1][1])
    print('\n')
    df_cm = pd.DataFrame(cm, range(2), range(2))
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
    plt.ylabel('Actual label', size = 20)
    plt.xlabel('Predicted label', size = 20)
    plt.xticks(np.arange(2), ['Fake', 'Real'], size = 16)
    plt.yticks(np.arange(2), ['Fake', 'Real'], size = 16)
    plt.ylim([2, 0])
    plt.show()
    
print_confusion_matrix(Y_val_org, np.argmax(model.predict(X),axis=1))